<a href="https://colab.research.google.com/github/MatheusFidelisPE/Projeto-AR/blob/main/projeto_1va_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/cap06/cap06-main.ipynb)

# Tema 8 - Estudo de parâmetros com ambientes diversos

Os algoritmos de aprendizagem possuem uma característica interessante, que é a capacidade de mudança de uma execução para outra após alterar minimamente um parâmetro do modelo. Assim, o trabalho atual busca testar diferentes valores para importantes parâmetros ** epsilon, número de passos, gamma, alfa e taxa de aprendizagem**.

Treinaremos o agente por **N** episódios, após isso faremos a execução de **M** episódios focando na recompensa obtida durante a execução dos M episódios. Para reduzir reduzir o fator aleatório, faremos a execução dos M episódios **X** vezes para calcular a média dessas recompensas e o desvio padrão.



## Configurações Iniciais

In [2]:
from IPython.display import clear_output
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # for saving videos
    !apt-get install ffmpeg

    !pip install gymnasium moviepy
    !pip install optuna

    # clone repository
    !git clone https://github.com/pablo-sampaio/rl_facil
    sys.path.append("/content/rl_facil")

else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )

clear_output()

In [2]:
# Imports importantes para criação dos algoritmos, exeução e apresentação dos gráficos e outros valores.
import gymnasium as gym
from gymnasium.wrappers import TimeLimit

import numpy as np

from cap06.nstep_sarsa import run_nstep_sarsa

from envs import RacetrackEnv
from envs.wrappers import ObservationDiscretizerWrapper

from util.experiments import repeated_exec
from util.plot import plot_result, plot_multiple_results
from util.notebook import display_videos_from_path
from util.qtable_helper import evaluate_qtable_policy, record_video_qtable

## Sarsa personalizado para treinar e testar

In [10]:
from util.qtable_helper import epsilon_greedy
from collections import deque
import gymnasium as gym
import numpy as np

def run_nstep_sarsa_by_eps(env, episodes, nsteps=1, lr=0.1, gamma=0.95, epsilon=0.1, verbose=False,**kargs):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)
    assert isinstance(nsteps, int)

    num_actions = env.action_space.n

    # inicializa a tabela Q com valores aleatórios pequenos (para evitar empates)
    if kargs.get("Q", None) is None:
      Q = np.random.uniform(low=-0.01, high=+0.01, size=(env.observation_space.n, num_actions))
    # Em caso de teste, a tabela Q já é carregada
    else:
      Q = kargs["Q"].copy()
    gamma_array = np.array([ gamma**i for i in range(0,nsteps)])
    gamma_power_nstep = gamma**nsteps

    # para cada episódio, guarda sua soma de recompensas (retorno não-descontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0

        state, _ = env.reset()
        # escolhe a próxima ação
        action = epsilon_greedy(Q, state, epsilon)

        # históricos de: estados, ações e recompensas
        hs = deque(maxlen=nsteps)
        ha = deque(maxlen=nsteps)
        hr = deque(maxlen=nsteps)

        # executa 1 episódio completo, fazendo atualizações na Q-table
        while not done:
            # realiza a ação
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            sum_rewards += reward

            # escolhe (antecipadamente) a ação do próximo estado
            next_action = epsilon_greedy(Q, next_state, epsilon)

            hs.append(state)
            ha.append(action)
            hr.append(reward)

            # se o histórico estiver completo com 'n' passos
            # vai fazer uma atualização no valor Q do estado mais antigo
            if len(hs) == nsteps:
                if terminated:
                    # para estados terminais
                    V_next_state = 0
                else:
                    # para estados não-terminais -- valor da próxima ação (já escolhida)
                    V_next_state = Q[next_state,next_action]

                # delta = (estimativa usando a nova recompensa) - estimativa antiga
                delta = ( sum(gamma_array * hr) + gamma_power_nstep * V_next_state ) - Q[hs[0],ha[0]]

                # atualiza a Q-table para o par (estado,ação) de n passos atrás
                Q[hs[0],ha[0]] += lr * delta

            # preparação para avançar mais um passo
            # lembrar que a ação a ser realizada já está escolhida
            state = next_state
            action = next_action
            # fim do laço por episódio

        # ao fim do episódio, atualiza o Q dos estados que restaram no histórico

        # é igual ao V_next_state, exceto em episódios muito curtos (com duração menor que "nsteps")
        V_end_state = 0 if terminated else Q[next_state,next_action]

        # inferior ao "nsteps" apenas em episódios muito curtos
        steps_to_end = min(nsteps, len(hs))
        for j in range(steps_to_end-1,0,-1):
            hs.popleft()
            ha.popleft()
            hr.popleft()
            delta = ( sum(gamma_array[0:j]*hr) + gamma_array[j]*V_end_state ) - Q[hs[0],ha[0]]
            Q[hs[0],ha[0]] += lr * delta

        sum_rewards_per_ep.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso
        if verbose and ((i+1) % 100 == 0):
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

    return sum_rewards_per_ep, Q

## Funções de execução de teste e treinamento
* vary_nsteps : Variação dos valores de passos
* vary_LR : Variação dos valores de LR
* vary_EPSILON: Variação dos valores de Epsilon
* vary_GAMMA: Variação dos valores de gamma






In [7]:
def vary_nsteps(env, values_list:list,number_of_executions, **kargs):

  #setup aplicação
  EPISODES = kargs["EPISODES"]
  LR = kargs["LR"]
  GAMMA = kargs["GAMMA"]
  EPSILON = kargs["EPSILON"]

  #Escolher imprimir ou não
  verbose = kargs["verbose"]

  #Estruturas de dados para armazenamento
  dict_of_each_parameter = dict()

  for nsteps in values_list:
    # Treinar o modelo
    dict_of_each_parameter[nsteps] = list()
    if verbose:
      print(f"Treinando com {nsteps} passos")
    rewards_training, Q = run_nstep_sarsa_by_eps(env, EPISODES, nsteps=nsteps, lr=LR, gamma=GAMMA, epsilon=EPSILON, verbose=verbose)
    for i in range(number_of_executions):
      if verbose:
        print(f"Rodando experimento {i+1} de {nsteps} passos")

      rewards_test, _ = run_nstep_sarsa_by_eps(env, EPISODES, nsteps=nsteps, lr=LR, gamma=GAMMA, epsilon=EPSILON, verbose=verbose, Q=Q)
      dict_of_each_parameter[nsteps].append(rewards_test)

  return dict_of_each_parameter


def vary_LR(env, values_list:list,number_of_executions, **kargs):

  #setup aplicação
  EPISODES = kargs["EPISODES"]
  GAMMA = kargs["GAMMA"]
  EPSILON = kargs["EPSILON"]
  #Escolher imprimir ou não
  verbose = kargs["verbose"]

  #Estruturas de dados para armazenamento
  dict_of_each_parameter = dict()

  for lr in values_list:
    # Treinar o modelo
    dict_of_each_parameter[lr] = list()
    if verbose:
      print(f"Treinando com {lr} passos")
    rewards_training, Q = run_nstep_sarsa_by_eps(env, EPISODES, nsteps=NSTEPS, lr=lr, gamma=GAMMA, epsilon=EPSILON, verbose=verbose)
    for i in range(number_of_executions):
      if verbose:
        print(f"Rodando experimento {i+1} de {lr} passos")

      rewards_test, _ = run_nstep_sarsa_by_eps(env, EPISODES, nsteps=NSTEPS, lr=lr, gamma=GAMMA, epsilon=EPSILON, verbose=verbose, Q=Q)
      dict_of_each_parameter[lr].append(rewards_test)



  return dict_of_each_parameter

def vary_EPSILON(env, values_list:list,number_of_executions, **kargs):

  #setup aplicação
  EPISODES = kargs["EPISODES"]
  GAMMA = kargs["GAMMA"]
  LR = kargs["LR"]
  #Escolher imprimir ou não
  verbose = kargs["verbose"]

  #Estruturas de dados para armazenamento
  dict_of_each_parameter = dict()

  for epsilon in values_list:
    # Treinar o modelo
    dict_of_each_parameter[epsilon] = list()
    if verbose:
      print(f"Treinando com {epsilon} passos")
    rewards_training, Q = run_nstep_sarsa_by_eps(env, EPISODES, nsteps=NSTEPS, lr=LR, gamma=GAMMA, epsilon=epsilon, verbose=verbose)
    for i in range(number_of_executions):
      if verbose:
        print(f"Rodando experimento {i+1} de {epsilon} passos")

      rewards_test, _ = run_nstep_sarsa_by_eps(env, EPISODES, nsteps=NSTEPS, lr=LR, gamma=GAMMA, epsilon=epsilon, verbose=verbose, Q=Q)
      dict_of_each_parameter[epsilon].append(rewards_test)



  return dict_of_each_parameter


def vary_GAMMA(env, values_list:list,number_of_executions, **kargs):
  #setup aplicação
  EPISODES = kargs["EPISODES"]
  LR = kargs["LR"]
  EPSILON = kargs["EPSILON"]
  #Escolher imprimir ou não
  verbose = kargs["verbose"]

  #Estruturas de dados para armazenamento
  dict_of_each_parameter = dict()

  for gamma in values_list:
    # Treinar o modelo
    dict_of_each_parameter[gamma] = list()
    if verbose:
      print(f"Treinando com {gamma} passos")
    rewards_training, Q = run_nstep_sarsa_by_eps(env, EPISODES, nsteps=NSTEPS, lr=LR, gamma=gamma, epsilon=EPSILON, verbose=verbose)
    for i in range(number_of_executions):
      if verbose:
        print(f"Rodando experimento {i+1} de {gamma} passos")

      rewards_test, _ = run_nstep_sarsa_by_eps(env, EPISODES, nsteps=NSTEPS, lr=LR, gamma=gamma, epsilon=EPSILON, verbose=verbose, Q=Q)
      dict_of_each_parameter[gamma].append(rewards_test)



  return dict_of_each_parameter


In [1]:
def experiment(env, env_name:str):
  rewards_gamma = vary_GAMMA(env, GAMMA_RAND, ITERATIONS_TO_CALCULATE_MEAN, LR=LR, EPSILON=EPSILON, verbose=True, EPISODES=EPISODES, NSTEPS=NSTEPS)
  rewards_epsilon = vary_EPSILON(env, EPSILON_RAND, ITERATIONS_TO_CALCULATE_MEAN, LR=LR,GAMMA=GAMMA, verbose=True, EPISODES=EPISODES, NSTEPS=NSTEPS)
  rewards_lr = vary_LR(env, LR_RAND, ITERATIONS_TO_CALCULATE_MEAN, GAMMA=GAMMA, EPSILON=EPSILON, verbose=True, EPISODES=EPISODES, NSTEPS=NSTEPS)
  rewards_nsteps = vary_nsteps(env, NSTEPS_RAND, ITERATIONS_TO_CALCULATE_MEAN, LR=LR, GAMMA=GAMMA, EPSILON=EPSILON, verbose=False, EPISODES=EPISODES)



## Execuções preliminares

In [8]:
# para ambientes gymnasium
#ENV_NAME, r_max = "Taxi-v3", 10
# ENV_NAME, r_max = "CliffWalking-v0", 0
# ENV_NAME, r_max = "FrozenLake-v1", 0
ENV_NAME, r_max = "RaceTrack-v0", 0

env = gym.make(ENV_NAME)


ITERATIONS_TO_CALCULATE_MEAN = 2
EPISODES = 1_0
EPISODES_TEST = 1_0

# Hyperparameters quando eles não são o foco do estudo
NSTEPS = 3
LR = 0.1
GAMMA = 0.95
EPSILON = 0.1

# Listas de Hyperparameters para que sejam executados os testes.
NSTEPS_RAND = [2**x for x in range(10)]
LR_RAND = np.random.uniform(low=0.1, high=1.0, size=(10,))
EPSILON_RAND = np.random.uniform(low=0.1, high=1.0, size=(10,))
GAMMA_RAND = np.random.uniform(low=0.1, high=0.95, size=(10,))

In [ ]:
rewards_gamma = vary_GAMMA(env, GAMMA_RAND, ITERATIONS_TO_CALCULATE_MEAN, LR=LR, EPSILON=EPSILON, verbose=True, EPISODES=EPISODES, NSTEPS=NSTEPS)

In [12]:
for key, value in rewards_gamma.items():
  print(f"#{key}\t Média:{np.mean(value)}\t\tDesvio Padrão: {np.std(value)}")

#0.10691801259342172	 Média:-112.35		Desvio Padrão: 44.39850785780982
#0.7567121610904911	 Média:-126.25		Desvio Padrão: 40.44363361519338
#0.2701062105859774	 Média:-120.2		Desvio Padrão: 36.289943510565024
#0.5880488108372625	 Média:-142.0		Desvio Padrão: 18.365728953678914
#0.8191887518623385	 Média:-126.0		Desvio Padrão: 40.65587288449235
#0.5698948091687034	 Média:-110.6		Desvio Padrão: 42.687703147393634
#0.6759190945123904	 Média:-110.35		Desvio Padrão: 42.490322427583436
#0.11335524548805287	 Média:-127.25		Desvio Padrão: 38.39905597797946
#0.7456100466078105	 Média:-134.05		Desvio Padrão: 26.369442542458117
#0.635376612995947	 Média:-122.2		Desvio Padrão: 42.011427016943856


In [14]:
rewards_nsteps = vary_nsteps(env, NSTEPS_RAND, ITERATIONS_TO_CALCULATE_MEAN, LR=LR, GAMMA=GAMMA, EPSILON=EPSILON, verbose=False, EPISODES=EPISODES)

In [15]:
for key, value in rewards_nsteps.items():
  print(f"#{key}\t Média:{np.mean(value)}\t\tDesvio Padrão: {np.std(value)}")

#1	 Média:-118.8		Desvio Padrão: 44.041571270789156
#2	 Média:-93.85		Desvio Padrão: 42.86639126401942
#4	 Média:-119.3		Desvio Padrão: 43.1
#8	 Média:-134.1		Desvio Padrão: 35.60182579587738
#16	 Média:-138.35		Desvio Padrão: 26.025516325329647
#32	 Média:-122.4		Desvio Padrão: 45.52845264227635
#64	 Média:-142.3		Desvio Padrão: 23.46294951620533
#128	 Média:-140.15		Desvio Padrão: 28.282989587382726
#256	 Média:-135.15		Desvio Padrão: 26.981984730556793
#512	 Média:-133.2		Desvio Padrão: 34.52622191899948


In [ ]:
rewards_lr = vary_LR(env, LR_RAND, ITERATIONS_TO_CALCULATE_MEAN, GAMMA=GAMMA, EPSILON=EPSILON, verbose=True, EPISODES=EPISODES, NSTEPS=NSTEPS)

In [57]:
for key, value in rewards_lr.items():
  print(f"#{key}\t Média:{np.mean(value)}\t\tDesvio Padrão: {np.std(value)}")

#0.7414197005512627	 Média:-124.65		Desvio Padrão: 34.84146236885014
#0.980994323727853	 Média:-126.45		Desvio Padrão: 34.43904034667633
#0.39459496814603645	 Média:-140.0		Desvio Padrão: 20.24598725673806
#0.9514998040758166	 Média:-129.45		Desvio Padrão: 33.7245830218848
#0.6917801118404305	 Média:-128.3		Desvio Padrão: 34.17908717329941
#0.41153183218705347	 Média:-116.15		Desvio Padrão: 41.27502271350072
#0.6895801178068364	 Média:-110.45		Desvio Padrão: 39.49110659376362
#0.13291738840305725	 Média:-135.6		Desvio Padrão: 28.24606167238187
#0.5624237690423813	 Média:-130.55		Desvio Padrão: 26.53389341954927
#0.41995191781471664	 Média:-124.65		Desvio Padrão: 40.50589463275685


In [ ]:
rewards_epsilon = vary_EPSILON(env, EPSILON_RAND, ITERATIONS_TO_CALCULATE_MEAN, LR=LR,GAMMA=GAMMA, verbose=True, EPISODES=EPISODES, NSTEPS=NSTEPS)

In [66]:
for key, value in rewards_epsilon.items():
  print(f"#{key}\t Média:{np.mean(value)}\t\tDesvio Padrão: {np.std(value)}")

#0.17406257093718114	 Média:-127.45		Desvio Padrão: 39.30582017971384
#0.48979584093646134	 Média:-110.55		Desvio Padrão: 48.08999376169641
#0.6053902548245806	 Média:-144.05		Desvio Padrão: 11.355505272774083
#0.21347623688434325	 Média:-123.45		Desvio Padrão: 31.853531986264883
#0.7926663288199893	 Média:-118.05		Desvio Padrão: 43.23710790513168
#0.9298904795823542	 Média:-121.65		Desvio Padrão: 36.904301917256205
#0.24862326088860406	 Média:-104.8		Desvio Padrão: 42.9902314485512
#0.6987369822818559	 Média:-123.95		Desvio Padrão: 40.54192274670751
#0.595112916305213	 Média:-131.3		Desvio Padrão: 30.990482409927083
#0.6343862394641873	 Média:-114.65		Desvio Padrão: 44.386118325440435


### Fim do Projeto
